In [6]:
import PySimpleGUI as sg
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import random
model_path= None
image=None
class_names=['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
left_column=sg.Column([
    [sg.Text('',key='class_real',text_color='red')],
    [sg.Image(key='image')]
], element_justification='left')# căn sát bên trái
right_column=sg.Column([
    [sg.Text('Classification result:'), sg.Text('   ',key='result'),
    sg.Text('Probability:'), sg.Text('   ',key='prob')],
    [sg.Graph((700,300),(0,0),(10,100),key='graph')]
],element_justification='left')# căn sát bên trái
layout=[
    [sg.Text('Choose your model',key='model_path_title')],
    [sg.Button('Choose model from browse',key='choose')],
    [sg.Text('Select an imagine from your computer or capture from webcam')],
    [sg.Button('Random',key='random'),sg.Button('Browse', key='file'),sg.Button('Capture',key='cam')],
    [sg.Button('Classify',key='classify'), sg.Button('Save',key='save')],
    [left_column,right_column]#Khối hiển thị gồm 2 cột
    ]
# layout=[
#     [sg.Text('Choose your model',key='model_path_title')],
#     [sg.Button('Choose model from browse',key='choose')],
#     [sg.Text('Select an imagine from your computer or capture from webcam')],
#     [sg.Button('Random',key='random'),sg.Button('Browse', key='file'),sg.Button('Capture',key='cam')],
#     [sg.Button('Classify',key='classify'), sg.Button('Save',key='save')],
#     [sg.Text('',key='class_real',text_color='red')],
#     [sg.Image(key='image')],
#     [sg.Text('Classification result:'), sg.Text('   ',key='result'),
#     sg.Text('Probability:'), sg.Text('   ',key='prob')],
#     [sg.Graph((200,200),(0,0),(10,100),key='graph')]
#     ]
window=sg.Window('CNN MNIST FASHION Demo', layout, size=(800,600),resizable=True)
while True:
    event,values=window.read()
    if event==sg.WIN_CLOSED:
        break
    if event=='choose':#Chọn mô đun CNN
        model_path=sg.popup_get_file('Enter the path to the CNN model', title='Load Model')
        if model_path is None:
            sg.popup_error('No model path entered')
            exit()
        else:
            try: 
                # with open(model_path, 'rb') as f: 
                #     model = pickle.load(f)
                model=tf.keras.models.load_model(model_path)
                window['model_path_title'].update(model_path)
            except:
                sg.popup_error('Invalid model path or format')
                exit()
    
    if event =='random':#Chọn ảnh ngẫu nhiên từ tập test fashion-mnist_test.csv
        try:
            test = pd.read_csv('fashion-mnist_test.csv')
            testing_array = np.array(test,dtype = 'float32')
            i = random.randrange(testing_array.shape[0])
            label = int(testing_array[i,0])
            window['class_real'].update(class_names[label],text_color='red')
            image = testing_array[i][1:].reshape(28,28)#hình 2 chiều
            image=cv2.resize(image,(400,400))
            window['image'].update(data=cv2.imencode('.png',image)[1].tobytes())
        except Exception as e:
            sg.popup_error(e)
            print(e)
    if event=='file':#Chọn ảnh trong browser
        file_path=sg.popup_get_file('Enter the path to your image', title='Upload Image')
        if file_path is not None:
            try:
                image=cv2.imread(file_path)
                image=cv2.resize(image,(400,400))
                window['image'].update(data=cv2.imencode('.png',image)[1].tobytes())
                window['class_real'].update('')
            except:
                sg.popup_error('Invalid file path or format')
        else:
            print(file_path)
            print(1)
    if event =='cam':#Dùng camera để bắt ảnh 
        cam=cv2.VideoCapture(0)
        ret, frame = cam.read()
        if ret:
            image=cv2.resize(frame,(400,400))
            window['image'].update(data=cv2.imencode('.png',image)[1].tobytes())
            window['class_real'].update('')
        cam.release()
    if event =='classify':#Phân loại ảnh
        if model_path is not None:
            try:
                row,col,*channel=image.shape
                
                if channel !=[]:
                    input_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                else:
                    input_image=image
                input_image=cv2.resize(input_image,(28,28))
                input_image=input_image.reshape(1,28,28,1)
                prediction=model.predict(input_image)#mảng xác suất [[A,B,C]]
                pred_class=np.argmax(prediction)#lấy chỉ số có giá trị xác suất cao
                pred_prob=prediction[0][pred_class]#giữ giá trị xác suất đó

                window['result'].update(class_names[pred_class],text_color='red')
                window['prob'].update(pred_prob,text_color='red')
                
                graph=window['graph']
                graph.erase()
                print(prediction)
                for i in range(10):
                    p=prediction[0][i]
                    # if p<0:
                    #     p=0
                    # else:
                    #     p=p/pred_prob
                    x=i+0.25
                    graph.draw_rectangle((x,0),(x+0.55,p*100),
                                        fill_color='gray')
                    
                    graph.draw_text(class_names[i], (x+0.25, 5), color='red')
                graph.Widget.configure(borderwidth=1, relief='solid')#vẽ viền đóng khung biểu đồ
            except Exception as e:
                sg.popup_error(e)
                print(e)
        else:
            sg.popup_error('Model is none')
    if event=='save':#Lưu lại hình ảnh file.png và kết quả phân loại file.txt
        try:
            file_name=sg.popup_get_text('Enter the file name to save', title='Save image and result')
            if file_name is not None:
                cv2.imwrite(file_name+'.png',image)
                with open(file_name+'.txt','w') as f:
                    f.write(f'Class: {window["result"].DisplayText}\n')
                    f.write(f'Probability: {window["prob"].DisplayText}\n')
                sg.popup('Image and result saved successfully') 
        except:
            sg.popup_error('No image selected or captured or invalid file name')
window.close()   

1/1 [==============================] - 0s 89ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
